In [1]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from paretoset import paretoset

from src.io import load_results

pd.options.plotting.backend = "plotly"
from src.scoring import get_score_df, get_eligible_points_df, get_summary_df

In [2]:
result_df = load_results()

score_df = get_score_df(result_df)
eligible_points_df = get_eligible_points_df(result_df)
summary_df = get_summary_df(result_df, score_df, eligible_points_df)
print(summary_df.round(2))

                   time  score  eligible_points  percentage
                                                           
baseline_random    0.00   4.37               10        0.44
saabas             0.03   5.04               10        0.50
tree_shap          0.02   6.06               10        0.61
sage               0.85   3.86                5        0.77
kernel_shap       21.22   9.13               10        0.91
lime              10.58   8.33               10        0.83
maple            545.88   8.00               10        0.80


In [3]:
min_x_axis_value = .01
summary_df.loc[summary_df.time < min_x_axis_value, 'time'] = min_x_axis_value
summary_df.percentage = summary_df.percentage * 100
summary_df['explainer_name'] = summary_df.index

In [4]:
mask = paretoset(summary_df[['percentage', 'time']], sense=["max", "min"])
pareto_df = summary_df[mask].sort_values('time')

In [7]:
fig = go.Figure()
fig = px.scatter(summary_df,
                 x='time',
                 y='percentage',
                 # s='explainer_name', # https://www.geeksforgeeks.org/how-to-annotate-matplotlib-scatter-plots/
                 # c='eligible_points',
                 text='explainer_name',
                 log_x=True,
                 labels={
                     "time": "Time (seconds)",
                     "percentage": "Percentage %",
                     'eligible_points': 'maximum score',
                     'explainer_name': 'Explainer'
                 },
                 title="Global performance of xAI methods",
                 )
fig.add_trace(go.Line(x=pareto_df.time,  #logx=True,
                      y=pareto_df.percentage,
                      text=pareto_df.explainer_name,
                      ))
fig.show();

C:\Users\kbelaid\Anaconda3\lib\site-packages\plotly\graph_objs\_deprecations.py:378: DeprecationWarning:

plotly.graph_objs.Line is deprecated.
Please replace it with one of the following more specific types
  - plotly.graph_objs.scatter.Line
  - plotly.graph_objs.layout.shape.Line
  - etc.




In [5]:
# todo choose options to have a more fair comparaison

# mention that sage is not comparable
# Pareto front